<a href="https://colab.research.google.com/github/hanseul1215/ESAA_study/blob/master/0516_recommend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1 추천시스템의 개요와 배경
사용자가 무엇을 원하는지 빠르게 찾아내 사용자의 선택 부담을 해결함  

추천 시스템을 구성하는데 사용하는 데이터
- 사용자가 어떤 상품을 구매했는가?
- 사용자가 어떤 제품을 Browse 했는가?
- 사용자가 무엇을 클릭했는가?
- 사용자의 제품 평점은?

추천 시스템의 유형
- 콘텐츠 기반 필터링
- 협업 필터링
> - 최근접 이웃 협업 필터링
> - 잠재요인 협업 필터링


# 2 콘텐츠 기반 필터링 추천 시스템
사용자가 특정한 아이템을 매우 선호하는 경우, 그 아이템과 비슷한 콘텐츠를 가진 다른 아이템을 추천하는 방식  
ex) 영화 -> 장르, 출연 배우, 감독, 영화 키워드 등의 콘텐츠가 유사한 다른 영화 추천

# 3 최근접 이웃 협업 필터링
사용자가 아이템에 매긴 평점 정보나 상품 구매 이력과 같은 사용자 행동 양식만을 기반으로 추천을 수행하는 것이 협업 필터링 방식
- 목표: 사용자-아이템 평점 매트릭스와 같은 축적된 사용자 행동 데이터를 기반으로 사용자가 아직 평가하지 않은 아이템을 예측 평가하는 것
- 사용자-아이템 평점 행렬 데이터에만 의지해 추천을 수행
> - 행은 개별 사용자, 열은 개별 아이템으로 구성
> - 다차원 희소행렬
- 사용자 기반 필터링, 아이템 기반 필터링으로 나뉨
- 코사인 유사도 -> 추천 시스템의 유사도 측정에 많이 적용

# 4 잠재 요인 협업 필터링
사용자-아이템 평점 매트릭스 속에 숨어 있는 잠재 요인을 추출해 추천 예측을 할 수 이께 하는 기법
- 대규모 다차원 행렬을 SVD와 같은 차원 감소 기법으로 분해하는 과정에서 잠재 요인을 추출(행렬 분해)
- 목표: 잠재요인을 기반으로 분해된 행렬의 내적을 통해 새로운 예측 사용자-아이템 평점 행렬 데이터를 만들어서 사용자가 아직 평점을 부여하지 않는 아이템에 대한 예측 평점을 생성함  

**행렬 분해**
- SVD, NMF가 있음
- M (사용자 행) * N (아이템 행) ---행렬 분해---> P행렬(M * K), Q.T행렬(K*M) (K: 잠재요인 차원 수)  

**확률적 경사하강법을 이용한 행렬 분해**  
P와 Q 행렬로 계산된 예측 R 행렬 값이 실제 R 행렬 값과 가장 최소의 오류를 가지도록 반복적인 비용 함수 최적화를 통해 P와 Q를 유추하는 것
1. P와 Q를 임의의 값을 가진 행렬로 설정
2. P와 Q.T 값을 곱해 예측 R 행렬 계산하고 예측 R 행렬과 실제 R 행렬에 해당하는 오류 값 계산
3. 이 오류를 최소화하는 P와 Q로 업데이트
4. 2, 3 반복하여 P,Q 값을 업데이트

In [1]:
import numpy as np

#원본 행렬 R 생성, 분해 행렬 P, Q 초기화, 잠재 요인 차원 K = 3
R = np.array([[4, np.NaN, np.NaN, 2, np.NaN,],
              [np.NaN, 5, np.NaN, 3, 1],
              [np.NaN, np.NaN, 3, 4, 4],
              [5, 2, 1, 1, np.NaN]])

num_users, num_items = R.shape
K = 3

np.random.seed(1)
P = np.random.normal(scale=1./K, size=(num_users, K))
Q = np.random.normal(scale=1./K, size=(num_items, K))

In [2]:
from sklearn.metrics import mean_squared_error

def get_rmse(R, P, Q, non_zeros):
    error = 0
    full_pred_matrix = np.dot(P, Q.T)

    #실제 R에서 널이 아닌 값의 위치 인덱스 추출해 실제 R 행렬과 예측 행렬의 RMSE 추출
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)

    return rmse

In [3]:
non_zeros = [(i, j, R[i, j]) for i in range(num_users) for j in range(num_items) if R[i, j] > 0]

steps = 1000
learning_rate = 0.01
r_lambda = 0.01
# SGD -> P, Q 계속 업데이트
for step in range(steps):
    for i, j, r in non_zeros:
        eij = r- np.dot(P[i, :], Q[j, :].T)

        P[i, :] = P[i, :] + learning_rate*(eij * Q[j, :] - r_lambda*P[i, :])
        Q[j, :] = Q[j, :] + learning_rate*(eij * P[i, :] - r_lambda*Q[j, :])

    rmse = get_rmse(R, P, Q, non_zeros)
    if (step % 50) == 0:
        print("### iteration step: ", step," rmse: ", rmse)

### iteration step:  0  rmse:  3.2011202085938044
### iteration step:  50  rmse:  0.6154267829430629
### iteration step:  100  rmse:  0.2536345481699502
### iteration step:  150  rmse:  0.12492903462065194
### iteration step:  200  rmse:  0.06275532232130014
### iteration step:  250  rmse:  0.03506384672845852
### iteration step:  300  rmse:  0.02389785844276385
### iteration step:  350  rmse:  0.019716500751376057
### iteration step:  400  rmse:  0.018154210025261398
### iteration step:  450  rmse:  0.017522410099519235
### iteration step:  500  rmse:  0.017227673409515896
### iteration step:  550  rmse:  0.01706152015024207
### iteration step:  600  rmse:  0.016947590721601208
### iteration step:  650  rmse:  0.016856507138095295
### iteration step:  700  rmse:  0.016776412354044923
### iteration step:  750  rmse:  0.016702292684141938
### iteration step:  800  rmse:  0.016631926279262455
### iteration step:  850  rmse:  0.01656427805855475
### iteration step:  900  rmse:  0.01649883

In [4]:
pred_matrix = np.dot(P, Q.T)
print('예측 행렬: \n', np.round(pred_matrix, 3))

예측 행렬: 
 [[3.991 1.    1.525 1.999 1.664]
 [7.926 4.978 1.8   2.98  1.003]
 [5.46  0.163 2.989 3.98  3.985]
 [4.969 2.007 0.999 1.023 0.365]]
